In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2


import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import the backtrader platform
import backtrader as bt


# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    # RSI parameters    
    params = (
    ('rsiLong', 20),
    ('rsiShort', 80),
    ('rsiPeriod', 14),
    ('printlog', False),
    )
    

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        
        # rsi
        self.rsiPeriod = self.params.rsiPeriod
        self.rsi = bt.indicators.RelativeStrengthIndex(period=self.rsiPeriod)
        
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # To keep track of pending orders
        self.order = None
        self.rsiLong = self.params.rsiLong
        self.rsiShort = self.params.rsiShort

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            # Buy if 14 day rsi less than 20    
            if self.rsi[0] < self.rsiLong:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                    

        else:
            # Sell if 7 day rsi over 80
            if self.rsi[0] > self.rsiShort:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(Curr Long Threshold %2d, Curr Short Threshold %2d, Curr RSI period %2d) Ending Value %.2f' % (self.rsiLong, self.rsiShort, self.rsiPeriod, self.broker.getvalue()), doprint=True)


In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

data = bt.feeds.PandasData(dataname = df['2016-01-01':'2018-01-01'])

# Add the Data Feed to Cerebro
cerebro.adddata(data)

cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Set commision to 1 cent
cerebro.broker.setcommission(commission=0.) 

# Set slippage
cerebro.broker.set_slippage_fixed(.01)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(volume=False, iplot=True)

In [ ]:
class RSIAnalyzer(bt.Analyzer):
    
    def __init__(self):
        
        self.rsiLong = self.strategy.params.rsiLong
        self.rsiShort = self.strategy.params.rsiShort
        self.rsiPeriod = self.strategy.params.rsiPeriod
        
    def start(self):
        self.init_cash = self.strategy.broker.cash 
        self.ntrade = 0

    def next(self):
        pass

    def notify_trade(self, trade):
        self.ntrade += 1
    
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        
        results = {
            'params': (self.rsiLong, self.rsiShort, self.rsiPeriod),
            'profit': self.final_val - self.init_cash,
            'ntrade': self.ntrade
        }
        
        return results

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
strats = cerebro.optstrategy(
    TestStrategy,
    rsiPeriod=(7,21),
    rsiLong=(10,20,25,30,35,40),
    rsiShort=(60,70,80,85,90))

# Analyzer: need to use months for comparison with out-of-sample
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(RSIAnalyzer, _name='RSIAnalyzer')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Set the commission
cerebro.broker.setcommission(commission=0.0)

#slippage
cerebro.broker.set_slippage_fixed(.01, 
                                  slip_open=True, 
                                  slip_limit=True, 
                                  slip_match=True, 
                                  slip_out=True)

# Run over everything
results = cerebro.run(maxcpus=1)

In [ ]:
params  = []
n_trades = []
profit   = []
for res in results:
    r = res[0].analyzers.RSIAnalyzer.get_analysis()
    params.append(r['params'])
    n_trades.append(r['ntrade'])
    profit.append(r['profit'])
    
    

fig, (prof, nt) = plt.subplots(1,2, figsize=(15,5))
_=prof.hist(profit, bins=np.arange(-5000,20000,1000))
_=nt.hist(n_trades, bins=np.arange(0,500,25))

prof.set_title('Profit Distribution')
nt.set_title('# of Trades Distribution')

In [ ]:
prof_ind = np.argmax(profit) 
best_params = params[prof_ind]
print("Best Parameters: ",best_params,"\nTotal Profit: ", profit[prof_ind], "\nNumber of Trades: ", n_trades[prof_ind])

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy,
                    rsiLong=40,
                    rsiShort=70,
                    rsiPeriod=21)


# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

#slippage
cerebro.broker.set_slippage_fixed(0.01)


#Analyzer
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(RSIAnalyzer, _name='RSIAnalyzer')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
in_samp_sharpe = results[0].analyzers.sharpe.get_analysis()
in_samp_drawdown = results[0].analyzers.drawdown.get_analysis()

In [ ]:
cerebro.plot(plotabove=True, iplot=True, volume=False)

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy,
                    rsiLong=40,
                    rsiShort=70,
                    rsiPeriod=21)


data = bt.feeds.PandasData(dataname = df['2018-01-01':'2019-01-01'])


# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

#slippage
cerebro.broker.set_slippage_fixed(0.01)


#Analyzer
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(RSIAnalyzer, _name='RSIAnalyzer')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results2 = cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
out_samp_sharpe = results2[0].analyzers.sharpe.get_analysis()
out_samp_drawdown = results2[0].analyzers.drawdown.get_analysis()

In [ ]:
cerebro.plot(plotabove=True, iplot=True, volume=False)

In [ ]:
print('in sample:')
print(in_samp_sharpe)
print('out sample:')
print(out_samp_sharpe)